# Data Preparation 📓✏️

> Before starting, define the **n-grams** to use.


In [42]:
# Only works with 2, 3, and 4
n_grams = 4

In [43]:
# loading shakespeare's works

import nltk
nltk.corpus.gutenberg.fileids()

caesar = nltk.corpus.gutenberg.words('shakespeare-caesar.txt')
print("Caesar length: ", len(caesar))
hamlet = nltk.corpus.gutenberg.words('shakespeare-hamlet.txt')
print("Hamlet length: ", len(hamlet))
macbeth = nltk.corpus.gutenberg.words('shakespeare-macbeth.txt')
print("Macbeth length: ", len(macbeth))

Caesar length:  25833
Hamlet length:  37360
Macbeth length:  23140


> Lowering the case of the text and removing punctuation


In [44]:
import string

# lowercase and remove punctuation
def preprocess_text(text):

    # Convert to lowercase
    text = [word.lower() for word in text]
    
    # Remove punctuation from each word
    text = [word.translate(str.maketrans('', '', string.punctuation)) for word in text]
    
    # Remove any empty strings that might result from words that were only punctuation
    text = [word for word in text if word]
    
    return text

# Preprocess texts
caesar_clean = preprocess_text(caesar)
hamlet_clean = preprocess_text(hamlet)
macbeth_clean = preprocess_text(macbeth)

print(caesar_clean[:10])
print(hamlet_clean[:10])
print(macbeth_clean[:10])

['the', 'tragedie', 'of', 'julius', 'caesar', 'by', 'william', 'shakespeare', '1599', 'actus']
['the', 'tragedie', 'of', 'hamlet', 'by', 'william', 'shakespeare', '1599', 'actus', 'primus']
['the', 'tragedie', 'of', 'macbeth', 'by', 'william', 'shakespeare', '1603', 'actus', 'primus']


> Creating the **list of n-grams** for each text


In [45]:
# list of n-grams for each text
def get_ngrams(tokens, n):
    return list(nltk.ngrams(tokens, n))

# Get n-grams for each text
caesar_ngrams = get_ngrams(caesar_clean, n_grams)
hamlet_ngrams = get_ngrams(hamlet_clean, n_grams)
macbeth_ngrams = get_ngrams(macbeth_clean, n_grams)

print(f"\n{n_grams}-grams from Caesar:")
print(caesar_ngrams[:5])
print(f"\n{n_grams}-grams from Hamlet:")
print(hamlet_ngrams[:5])
print(f"\n{n_grams}-grams from Macbeth:")
print(macbeth_ngrams[:5])


4-grams from Caesar:
[('the', 'tragedie', 'of', 'julius'), ('tragedie', 'of', 'julius', 'caesar'), ('of', 'julius', 'caesar', 'by'), ('julius', 'caesar', 'by', 'william'), ('caesar', 'by', 'william', 'shakespeare')]

4-grams from Hamlet:
[('the', 'tragedie', 'of', 'hamlet'), ('tragedie', 'of', 'hamlet', 'by'), ('of', 'hamlet', 'by', 'william'), ('hamlet', 'by', 'william', 'shakespeare'), ('by', 'william', 'shakespeare', '1599')]

4-grams from Macbeth:
[('the', 'tragedie', 'of', 'macbeth'), ('tragedie', 'of', 'macbeth', 'by'), ('of', 'macbeth', 'by', 'william'), ('macbeth', 'by', 'william', 'shakespeare'), ('by', 'william', 'shakespeare', '1603')]


> Counting the **frequency** of each ngram's subsequent token and the own ngram's frequency


In [46]:
# Count the frequency of each n-gram's subsequent token
# Example: from_ngram_to_next_token_counts[('to', 'be')] = {'count': 15, 'next_tokens': {'or': 10, 'not': 5}}
# This means that the n-gram 'to be' is followed by 'or' 10 times and by 'not' 5 times, and the total count of ngram 'to be' is 15


from_ngram_to_next_token_counts = {}

def count_next_token(ngrams, tokens, dictionary, n):
    for i in range(len(ngrams)):
        if i < len(tokens) - n:  # Make sure we have a next token
            ngram = ngrams[i]
            next_token = tokens[i + n]  # Get the token that follows the ngram
            
            # If ngram doesn't exist, create new entry with count and next_tokens dictionary
            if ngram not in dictionary:
                dictionary[ngram] = {
                    'count': 0,
                    'next_tokens': {}
                }
            
            # Increment total count for this ngram
            dictionary[ngram]['count'] += 1
            
            # Add or increment next token count
            if next_token not in dictionary[ngram]['next_tokens']:
                dictionary[ngram]['next_tokens'][next_token] = 1
            else:
                dictionary[ngram]['next_tokens'][next_token] += 1

# Count next tokens from each play
count_next_token(caesar_ngrams, caesar_clean, from_ngram_to_next_token_counts, n_grams)
count_next_token(hamlet_ngrams, hamlet_clean, from_ngram_to_next_token_counts, n_grams)
count_next_token(macbeth_ngrams, macbeth_clean, from_ngram_to_next_token_counts, n_grams)

# Testing function depending on number of ngrams
if n_grams == 2:

    ngram = ('to', 'be')
    result = from_ngram_to_next_token_counts[ngram]
    print(f"N-gram '{ngram[0]} {ngram[1]}':")
    print(f"Total count: {result['count']}")
    print(f"Next tokens: {result['next_tokens']}")

    print("\nRaw data:")
    print(from_ngram_to_next_token_counts[('to', 'be')])

elif n_grams == 3:
    ngram = ('to', 'be', 'or')
    result = from_ngram_to_next_token_counts[ngram]
    print(f"N-gram '{ngram[0]} {ngram[1]} {ngram[2]}':")
    print(f"Total count: {result['count']}")
    print(f"Next tokens: {result['next_tokens']}")

    print("\nRaw data:")
    print(from_ngram_to_next_token_counts[('to', 'be', 'or')])

elif n_grams == 4:
    ngram = ('to', 'be', 'or', 'not')
    result = from_ngram_to_next_token_counts[ngram]
    print(f"N-gram '{ngram[0]} {ngram[1]} {ngram[2]} {ngram[3]}':")
    print(f"Total count: {result['count']}")
    print(f"Next tokens: {result['next_tokens']}")

    print("\nRaw data:")
    print(from_ngram_to_next_token_counts[('to', 'be', 'or', 'not')])

N-gram 'to be or not':
Total count: 1
Next tokens: {'to': 1}

Raw data:
{'count': 1, 'next_tokens': {'to': 1}}


# Probability Distribution 📊

> Calculating the **probability** of each n-gram's subsequent token

In [47]:
# Calculating the probability of each ngram's subsequent token

from_ngram_to_next_token_probs = {}

def calculate_probabilities(counts_dict):

    probs_dict = {}
    
    # For each ngram in the counts dictionary
    for ngram, data in counts_dict.items():
        total_count = data['count']
        next_tokens = data['next_tokens']
        
        # Calculate probability for each next token
        probs = {}
        for token, count in next_tokens.items():
            prob = count / total_count
            probs[token] = round(prob, 3)  # Round to 3 decimal places
            
        probs_dict[ngram] = probs
    
    return probs_dict

# Calculate probabilities
from_ngram_to_next_token_probs = calculate_probabilities(from_ngram_to_next_token_counts)


# Examples of probabilities depending on ngrams

if n_grams == 2:

    # Example probabilities for the same ngram we checked before
    ngram = ('to', 'be')
    print(f"Probabilities for ngram '{ngram[0]} {ngram[1]}':")
    print(from_ngram_to_next_token_probs[ngram])

    print("\nMore examples:")
    for ngram, probs in list(from_ngram_to_next_token_probs.items())[:5]:
        print(f"\nngram '{ngram[0]} {ngram[1]}':")
        print(f"Probabilities: {probs}")

elif n_grams == 3:

    # Example probabilities for the same ngram we checked before
    ngram = ('to', 'be', 'or')
    print(f"Probabilities for ngram '{ngram[0]} {ngram[1]} {ngram[2]}':")
    print(from_ngram_to_next_token_probs[ngram])

    print("\nMore examples:")
    for ngram, probs in list(from_ngram_to_next_token_probs.items())[:5]:
        print(f"\nngram '{ngram[0]} {ngram[1]} {ngram[2]}':")
        print(f"Probabilities: {probs}")

elif n_grams == 4:

    # Example probabilities for the same ngram we checked before
    ngram = ('to', 'be', 'or', 'not')
    print(f"Probabilities for ngram '{ngram[0]} {ngram[1]} {ngram[2]} {ngram[3]}':")
    print(from_ngram_to_next_token_probs[ngram])

    print("\nMore examples:")
    for ngram, probs in list(from_ngram_to_next_token_probs.items())[:5]:
        print(f"\nngram '{ngram[0]} {ngram[1]} {ngram[2]} {ngram[3]}':")
        print(f"Probabilities: {probs}")

Probabilities for ngram 'to be or not':
{'to': 1.0}

More examples:

ngram 'the tragedie of julius':
Probabilities: {'caesar': 1.0}

ngram 'tragedie of julius caesar':
Probabilities: {'by': 1.0}

ngram 'of julius caesar by':
Probabilities: {'william': 1.0}

ngram 'julius caesar by william':
Probabilities: {'shakespeare': 1.0}

ngram 'caesar by william shakespeare':
Probabilities: {'1599': 1.0}


# Sampling Next Token 🎲


> Sampling the next token **based on the probability distribution** for a given ngram.


In [48]:
import numpy as np

def sample_next_token(ngram, prob_dict):
    """
    Sample the next token based on the probability distribution for a given ngram.
    
    Args:
        ngram (tuple): A tuple of tokens (previous tokens)
        prob_dict (dict): Dictionary containing probability distributions for ngrams
    
    Returns:
        str: The sampled next token
        If ngram not found, returns None
    """
    # Check if ngram exists in our probability dictionary
    if ngram not in prob_dict:
        return None
    
    # Get probability distribution for this ngram
    next_token_probs = prob_dict[ngram]
    
    # Get tokens and their probabilities as separate lists
    tokens = list(next_token_probs.keys())
    probs = list(next_token_probs.values())
    
    # Normalize probabilities to ensure they sum to 1
    probs_sum = sum(probs)
    if probs_sum > 0:
        probs = [p/probs_sum for p in probs]
    
    # Sample one token based on the probability distribution
    next_token = np.random.choice(tokens, p=probs)
    
    return next_token


# Test the sampling function depending on ngram
if n_grams == 2:

    test_ngram = ('to', 'be')
    print(f"Testing sampling for ngram '{test_ngram[0]} {test_ngram[1]}'")
    print(f"Probability distribution: {from_ngram_to_next_token_probs[test_ngram]}")

    # Sample multiple times to see the distribution
    n_samples = 1000
    samples = [sample_next_token(test_ngram, from_ngram_to_next_token_probs) for _ in range(n_samples)]

    # Calculate and print the empirical distribution
    unique_tokens = set(samples)
    empirical_dist = {token: samples.count(token)/n_samples for token in unique_tokens}

    print(f"\nEmpirical distribution after {n_samples} samples:")
    for token, prob in empirical_dist.items():
        print(f"'{token}': {prob:.3f}")

if n_grams == 3:

    test_ngram = ('to', 'be', 'or')
    print(f"Testing sampling for ngram '{test_ngram[0]} {test_ngram[1]} {test_ngram[2]}'")
    print(f"Probability distribution: {from_ngram_to_next_token_probs[test_ngram]}")

    # Sample multiple times to see the distribution
    n_samples = 1000
    samples = [sample_next_token(test_ngram, from_ngram_to_next_token_probs) for _ in range(n_samples)]

    # Calculate and print the empirical distribution
    unique_tokens = set(samples)
    empirical_dist = {token: samples.count(token)/n_samples for token in unique_tokens}

    print(f"\nEmpirical distribution after {n_samples} samples:")
    for token, prob in empirical_dist.items():
        print(f"'{token}': {prob:.3f}")

if n_grams == 4:

    test_ngram = ('to', 'be', 'or', 'not')
    print(f"Testing sampling for ngram '{test_ngram[0]} {test_ngram[1]} {test_ngram[2]} {test_ngram[3]}'")
    print(f"Probability distribution: {from_ngram_to_next_token_probs[test_ngram]}")

    # Sample multiple times to see the distribution
    n_samples = 1000
    samples = [sample_next_token(test_ngram, from_ngram_to_next_token_probs) for _ in range(n_samples)]

    # Calculate and print the empirical distribution
    unique_tokens = set(samples)
    empirical_dist = {token: samples.count(token)/n_samples for token in unique_tokens}

    print(f"\nEmpirical distribution after {n_samples} samples:")
    for token, prob in empirical_dist.items():
        print(f"'{token}': {prob:.3f}")

Testing sampling for ngram 'to be or not'
Probability distribution: {'to': 1.0}

Empirical distribution after 1000 samples:
'to': 1.000


# Generating Text 📝


> Generating text starting from an n-gram and a specified amount of words

In [49]:
def generate_text_from_ngram(initial_ngram, num_words, prob_dict):
    """
    Generate text starting from an initial n-gram.
    
    Args:
        initial_ngram (tuple): The starting n-gram
        num_words (int): Number of words to generate
        prob_dict (dict): Dictionary containing probability distributions
    
    Returns:
        str: Generated text
    """
    # Initialize text with the initial n-gram
    generated_words = list(initial_ngram)
    
    # Generate remaining words
    current_ngram = initial_ngram
    for _ in range(num_words - len(initial_ngram)):
        # Sample next token
        next_token = sample_next_token(current_ngram, prob_dict)
        
        # If we can't continue (no following tokens found), break
        if next_token is None:
            break
            
        # Add the new token to our generated text
        generated_words.append(next_token)
        
        # Create new n-gram for next iteration
        current_ngram = tuple(generated_words[-n_grams:])
    
    # Join all words with spaces
    return ' '.join(generated_words)

if n_grams == 2:

    # Test the text generation with different initial n-grams
    test_cases = [
        ('to', 'be'),
        ('truly', 'you'),
        ('is', 'this')
    ]

    print("Generated Text Examples:\n")
    for initial_ngram in test_cases:
        print(f"Starting with '{initial_ngram[0]} {initial_ngram[1]}':")
        generated_text = generate_text_from_ngram(initial_ngram, 20, from_ngram_to_next_token_probs)
        print(f"{generated_text}\n")

if n_grams == 3:

    # Test the text generation with different initial n-grams
    test_cases = [
        ('to', 'be', 'or'),
        ('truly', 'you', 'were'),
        ('is', 'this', 'a')
    ]

    print("Generated Text Examples:\n")
    for initial_ngram in test_cases:
        print(f"Starting with '{initial_ngram[0]} {initial_ngram[1]}':")
        generated_text = generate_text_from_ngram(initial_ngram, 20, from_ngram_to_next_token_probs)
        print(f"{generated_text}\n")

if n_grams == 4:

    # Test the text generation with different initial n-grams
    test_cases = [
        ('to', 'be', 'or', 'not'),
        ('truly', 'you', 'were', 'not'),
        ('is', 'this', 'a', 'dagger')
    ]

    print("Generated Text Examples:\n")
    for initial_ngram in test_cases:
        print(f"Starting with '{initial_ngram[0]} {initial_ngram[1]}':")
        generated_text = generate_text_from_ngram(initial_ngram, 20, from_ngram_to_next_token_probs)
        print(f"{generated_text}\n")

Generated Text Examples:

Starting with 'to be':
to be or not to be that is the question whether tis nobler in the minde to suffer the slings

Starting with 'truly you':
truly you were not

Starting with 'is this':
is this a dagger which i see before me the handle toward my hand come let me clutch thee i



# **Results** 🏁

This examples are generted using 2, 3 and 4 ngrams with a longitude of 20 words.

### **2-grams**

**Starting with 'to be':**
to be wrencht with an angry god macd i haue an immediate freedome of repeale caes what can the deuill

**Starting with 'truly you':**
truly you were sent for or no for he was not ambitious 1 if it be no assistant for a

**Starting with 'is this':**
is this that i must fall downe or else this braine of mine but vertue as it behoues my daughter

### **3-grams**

**Starting with 'to be':**
to be or not to crack the winde of the poore phrase roaming it thus you l tender me a

**Starting with 'truly you':**
truly you were best cin what is my lord ham how say you then would heart of man once think

**Starting with 'is this':**
is this a dagger which i see before me the handle toward my hand come let me wipe thy face


### **4-grams**

**Starting with 'to be':**
to be or not to be that is the question whether tis nobler in the minde to suffer the slings

**Starting with 'truly you':**
truly you were not

**Starting with 'is this':**
is this a dagger which i see before me the handle toward my hand come let me clutch thee i

# **Analysis** 🔍

The results from text generation using 2, 3, and 4-grams show a clear pattern in terms of coherence and fluency. Below is a brief analysis of each:

### **2-grams**

- The generated text lacks strong coherence, often forming disjointed phrases.

- There are abrupt shifts in meaning, making it difficult to follow a logical sequence.

- Some phrases resemble Shakespearean language but appear fragmented.

### **3-grams**

- Shows slight improvement in sentence structure compared to 2-grams.

- Some phrases start to resemble coherent speech but still contain inconsistencies.

- There are recognizable Shakespearean phrases, though they are often incomplete.

### **4-grams**

- Produces the most coherent and meaningful text.

- Famous phrases, such as "to be or not to be" and "is this a dagger which I see before me", emerge naturally.

- The output maintains better fluency and preserves original phrasing from Shakespearean texts.


> As expected, **increasing the n-gram size improves** text coherence and fluency. While 2-grams produce fragmented and disjointed text, 4-grams generate recognizable and structured Shakespearean phrases. This suggests that **higher n-grams help retain more meaningful context, though they may still lack true creativity** or deeper understanding of language.





# **Tests** 🧪

Testing that all functions work as intended

In [50]:
import unittest
from collections import defaultdict

class TestShakespeareGenerator(unittest.TestCase):
    def setUp(self):
        """Set up test data"""
        self.test_tokens = ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
        self.test_ngrams = list(nltk.ngrams(self.test_tokens, n_grams))
        self.test_counts = {}
        self.test_probs = {}
        
    def test_preprocess_text(self):
        """Test text preprocessing"""
        test_text = ['Hello,', 'World!', 'Test.']
        processed = preprocess_text(test_text)
        self.assertEqual(processed, ['hello', 'world', 'test'])
        
    def test_get_ngrams(self):
        """Test n-gram generation"""
        ngrams = get_ngrams(self.test_tokens, n_grams)
        self.assertTrue(isinstance(ngrams, list))
        self.assertTrue(all(len(gram) == n_grams for gram in ngrams))
        
    def test_count_next_token(self):
        """Test token counting"""
        count_next_token(self.test_ngrams, self.test_tokens, self.test_counts, n_grams)
        
        # Check structure of counts dictionary
        for ngram in self.test_counts:
            self.assertIn('count', self.test_counts[ngram])
            self.assertIn('next_tokens', self.test_counts[ngram])
            self.assertTrue(isinstance(self.test_counts[ngram]['count'], int))
            self.assertTrue(isinstance(self.test_counts[ngram]['next_tokens'], dict))
            
    def test_calculate_probabilities(self):
        """Test probability calculation"""
        # First generate counts
        count_next_token(self.test_ngrams, self.test_tokens, self.test_counts, n_grams)
        
        # Calculate probabilities
        self.test_probs = calculate_probabilities(self.test_counts)
        
        # Check probabilities sum to 1 (or close to 1 due to rounding)
        for ngram in self.test_probs:
            prob_sum = sum(self.test_probs[ngram].values())
            self.assertAlmostEqual(prob_sum, 1.0, places=2)
            
    def test_sample_next_token(self):
        """Test token sampling"""
        # First generate probabilities
        count_next_token(self.test_ngrams, self.test_tokens, self.test_counts, n_grams)
        self.test_probs = calculate_probabilities(self.test_counts)
        
        # Test sampling
        if len(self.test_ngrams) > 0:
            test_ngram = self.test_ngrams[0]
            sampled_token = sample_next_token(test_ngram, self.test_probs)
            
            # Check if sampled token is in the possible next tokens
            if sampled_token is not None:
                self.assertIn(sampled_token, self.test_probs[test_ngram])
                
    def test_generate_text(self):
        """Test text generation"""
        if len(self.test_ngrams) > 0:
            initial_ngram = self.test_ngrams[0]
            generated_text = generate_text_from_ngram(initial_ngram, 10, self.test_probs)
            
            # Check if generated text is a string and contains words
            self.assertTrue(isinstance(generated_text, str))
            self.assertTrue(len(generated_text.split()) > 0)
            
    def test_shakespeare_specific(self):
        """Test with actual Shakespeare data"""
        # Test specific Shakespeare n-grams
        if n_grams == 2:
            test_ngram = ('to', 'be')
        elif n_grams == 3:
            test_ngram = ('to', 'be', 'or')
        elif n_grams == 4:
            test_ngram = ('to', 'be', 'or', 'not')
            
        # Check if the famous phrases are in our data
        self.assertIn(test_ngram, from_ngram_to_next_token_probs)
        
        # Generate text from famous phrase
        generated = generate_text_from_ngram(test_ngram, 10, from_ngram_to_next_token_probs)
        self.assertTrue(isinstance(generated, str))
        self.assertTrue(len(generated.split()) > 0)

# Run the tests
if __name__ == '__main__':
    test_suite = unittest.TestLoader().loadTestsFromTestCase(TestShakespeareGenerator)
    unittest.TextTestRunner(verbosity=2).run(test_suite)

test_calculate_probabilities (__main__.TestShakespeareGenerator.test_calculate_probabilities)
Test probability calculation ... ok
test_count_next_token (__main__.TestShakespeareGenerator.test_count_next_token)
Test token counting ... ok
test_generate_text (__main__.TestShakespeareGenerator.test_generate_text)
Test text generation ... ok
test_get_ngrams (__main__.TestShakespeareGenerator.test_get_ngrams)
Test n-gram generation ... ok
test_preprocess_text (__main__.TestShakespeareGenerator.test_preprocess_text)
Test text preprocessing ... ok
test_sample_next_token (__main__.TestShakespeareGenerator.test_sample_next_token)
Test token sampling ... ok
test_shakespeare_specific (__main__.TestShakespeareGenerator.test_shakespeare_specific)
Test with actual Shakespeare data ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.030s

OK
